In [1]:
# Dataset e-commerce complet
ecommerce_data = [
    {
        "user_id": "U001", "name": "Alice", "age": 25, "city": "Paris",
        "orders": [
            {"order_id": "O1", "date": "2024-01", "items": [("laptop", 999, 1), ("mouse", 25, 2)], "status": "delivered"},
            {"order_id": "O2", "date": "2024-02", "items": [("keyboard", 75, 1)], "status": "pending"},
            {"order_id": "O3", "date": "2024-03", "items": [("monitor", 300, 1), ("webcam", 50, 1)], "status": "delivered"},
            {"order_id": "O4", "date": "2024-04", "items": [("monitor", 300, 1), ("pad", 15, 1)], "status": "delivered"}
        ]
    },
    {
        "user_id": "U002", "name": "Bob", "age": 35, "city": "Lyon", 
        "orders": [
            {"order_id": "O4", "date": "2024-01", "items": [("laptop", 999, 1)], "status": "delivered"},
            {"order_id": "O5", "date": "2024-02", "items": [("mouse", 25, 1), ("pad", 15, 3)], "status": "delivered"}
        ]
    },
    {
        "user_id": "U003", "name": "Charlie", "age": 28, "city": "Paris",
        "orders": [
            {"order_id": "O6", "date": "2024-01", "items": [("keyboard", 75, 2), ("mouse", 25, 1)], "status": "cancelled"},
            {"order_id": "O7", "date": "2024-03", "items": [("laptop", 999, 1), ("monitor", 300, 2)], "status": "delivered"}
        ]
    }
]




Extrayez tous les produits achetés par des utilisateurs parisiens avec quantité > 1

In [14]:
prdt_user_paris = [(data['name'], item[0], item[2]) for data in ecommerce_data 
                   for order in data['orders'] for item in order['items']
                   if data['city'] == 'Paris' and item[2] > 1 and order["status"] == "delivered"]

In [15]:
prdt_user_paris

[('Alice', 'mouse', 2), ('Charlie', 'monitor', 2)]

Calculez le "score de fidélité" par utilisateur: nombre_commandes * 10 + total_dépensé * 0.1, mais seulement pour les utilisateurs avec au moins 2 commandes livrées.

In [12]:
score_fide = [
    (
        data["name"],
        sum(1 for order in data["orders"] if order["status"] == "delivered") * 10 + sum(
            sum(price * quantity for _, price, quantity in order["items"])
            for order in data["orders"] if order["status"] == "delivered"
        ) * 0.1
    )
    for data in ecommerce_data
    if sum(1 for order in data["orders"] if order["status"] == "delivered") >= 2
]

In [13]:
score_fide

[('Alice', 201.4), ('Bob', 126.9)]

Trouvez les utilisateurs qui ont commandé le même produit dans différentes commandes (détection de récurrence d'achat).

In [42]:
users_with_recurrent = {}
for user in ecommerce_data:
    user_id = user["user_id"]
    name = user["name"]
    product_orders = {}  # Dictionnaire pour compter dans combien de commandes un produit apparaît
    
    # Parcourir les commandes de l'utilisateur
    for order in user["orders"]:  # Corrigé : user["orders"] au lieu de orders
        # Parcourir les items de chaque commande
        for item in order["items"]:
            produit = item[0]  # Nom du produit
            product_orders[produit] = product_orders.get(produit, 0) + 1
    
    # Vérifier les produits qui apparaissent dans plusieurs commandes
    recurrent_products = [produit for produit, count in product_orders.items() if count > 1]
    if recurrent_products:  # Si des produits sont récurrents
        users_with_recurrent[user_id] = {"name": name, "products": recurrent_products}

# Afficher les résultats
for user_id, info in users_with_recurrent.items():
    print(f"Utilisateur {info['name']} a acheté les produits suivants dans plusieurs commandes :")
    for produit in info["products"]:
        print(f"  - {produit} {count}")

Utilisateur Alice a acheté les produits suivants dans plusieurs commandes :
  - monitor 2


In [27]:
# Étape 1 : Identifier tous les produits uniques
all_products = set()
for user in ecommerce_data:
    for order in user['orders']:
        for item in order['items']:
            all_products.add(item[0])
all_products = sorted(list(all_products))  # Trier pour un ordre cohérent

# Étape 2 : Créer la matrice
prdt_user = []
for user in ecommerce_data:
    # Dictionnaire pour compter les quantités par produit pour cet utilisateur
    quantities = {product: 0 for product in all_products}
    for order in user['orders']:
        for item in order['items']:
            product, _, quantity = item
            quantities[product] += quantity
    # Convertir en liste alignée sur all_products
    prdt_user.append([quantities[product] for product in all_products])

# Afficher la matrice avec les noms des utilisateurs et produits
print("Produits :", all_products)
for i, user in enumerate(ecommerce_data):
    print(f"{user['name']} : {prdt_user[i]}")

Produits : ['keyboard', 'laptop', 'monitor', 'mouse', 'pad', 'webcam']
Alice : [1, 1, 2, 2, 1, 1]
Bob : [0, 1, 0, 1, 3, 0]
Charlie : [2, 1, 2, 1, 0, 0]


In [28]:
# Dataset réseau social
social_network = [
    {"user": "Alice", "posts": [{"content": "Hello world", "likes": 15, "tags": ["tech", "python"]}, 
                                {"content": "Data science rocks", "likes": 23, "tags": ["data", "ml"]}],
     "following": ["Bob", "Diana"], "followers": ["Bob", "Charlie", "Eve"]},
    {"user": "Bob", "posts": [{"content": "Morning coffee", "likes": 8, "tags": ["life"]},
                              {"content": "Python tutorial", "likes": 45, "tags": ["tech", "python", "tutorial"]}],
     "following": ["Alice", "Charlie"], "followers": ["Alice", "Frank"]},
    {"user": "Charlie", "posts": [{"content": "Weekend vibes", "likes": 12, "tags": ["life", "weekend"]}],
     "following": ["Alice"], "followers": ["Alice", "Bob"]}
]

In [39]:
# Initialiser le dictionnaire des connexions mutuelles
mutual_connections = {}

# Parcourir chaque utilisateur A
for user_A in social_network:
    mutual_list = []  # Liste pour stocker les connexions mutuelles de A
    user_A_name = user_A["user"]  # Nom de l'utilisateur A
    following = user_A["following"]  # Personnes suivies par A
    
    # Parcourir chaque personne B suivie par A
    for person_B in following:
        # Trouver l'utilisateur B dans social_network
        for user_B in social_network:
            if user_B["user"] == person_B:
                # Vérifier si A est dans les followers de B
                if user_A_name in user_B["followers"]:
                    mutual_list.append(person_B)
                break  # Sortir une fois B trouvé
    
    # Ajouter l'entrée au dictionnaire
    mutual_connections[user_A_name] = mutual_list

# Afficher le résultat
print(mutual_connections)

{'Alice': ['Bob'], 'Bob': ['Alice', 'Charlie'], 'Charlie': ['Alice']}
